In [ ]:
# ejecutamos una orden simple (input) y vemos la respuesta (output). Todo lo que empice con # no se interpreta
2 + 2

In [ ]:
#guardamos objeto en una variable
a = 2

In [ ]:
#vemos la variable
a

In [ ]:
#la utilizamos luego
a + 3

In [ ]:
#si cambiamos la variable esta se modifica
a = 5

In [ ]:
a + 3

In [ ]:
# instalo geopandas 
!pip install geopandas

In [ ]:
# instalo mapclassify
!pip install mapclassify

In [ ]:
# importo librerias
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# descargo los datos
!git clone https://github.com/alephcero/untref_analisis_espacial_2020.git

In [ ]:
radios = gpd.read_file('/content/untref_analisis_espacial_2020/carto/caba_radios_censales.geojson')

In [ ]:
#visualizo los primeros registros de mi variable radios
radios.head()

In [ ]:
#realizo un primer plot sencillo de mi shape
radios.plot()

In [ ]:
#realizo otro plot sin especificar algun criterio de visualizacion
radios.plot(column = "POBLACION")

In [ ]:
#realizo un plot especificando criterios de visualizacion de 10 quintiles 
radios.plot(column = "POBLACION", scheme = "Quantiles", k = 10)

In [ ]:
#reproyecto mi cartografia a una proyeccion en metros. Pueden leer mas en: http://epsg.io/3857
radios_metros = radios.to_crs(epsg = 3857)
radios_metros.head()

In [ ]:
#los ejes de coordenadas ya nos nuestran un cambio de unidad de medida
radios_metros.plot()

In [ ]:
#obtenemos el area de cada radio censal al extraer la PROPIEDAD geografica area de nuestra columna geometry. Area no lleva () porque es una propiedad, no una funcion. Veremos esto mas adelante.
radios_metros["area_m"] = radios_metros.geometry.area

In [ ]:
#como nuestra proyeccion esta en metros y queremos pasarlo a km2, debemos dividir nuestra columna area en metros (area_m) por 1000000
radios_metros["area_km"] = radios_metros.area_m / 1000000

In [ ]:
#finalmente dividimos nuestra poblacion por el area en km2
radios_metros['densidad'] = radios_metros.POBLACION / radios_metros.area_km

radios_metros.head()

In [ ]:
#ploteamos los resultados
radios_metros.plot(column = "densidad",scheme = "Quantiles", k = 10)

In [ ]:
# podemos visualizar menos datos, solo algunas columnas 
radios_metros.loc[:,['RADIO_ID','densidad']].head()

In [ ]:
# o solo algunas filas
radios_metros.loc[:7,['RADIO_ID','densidad']]

In [ ]:
#podemos hacer esto mismo seleccionando valores de la variable
radios_metros.loc[radios_metros.BARRIO == 'RETIRO',['RADIO_ID','BARRIO','densidad']]

In [ ]:
# vemos que tanto al principio como al final todos los registros (91 en este caso) son de retiro. 
# pero la ventaja de datos espaciales es que podemos visualizarlos y ver si obtuvimos lo que queriamos
# AGREGAMOS LA VARIABLE GEOMETRY, ES IMPORTANTE!

radios_metros.loc[radios_metros.BARRIO == 'RETIRO',['RADIO_ID','BARRIO','densidad','geometry']].plot()

In [ ]:
# vamos a seleccionar radios de la comuna 4 y ver de obtener algunos estadisticos. 
# Asi vemos que CANTIDAD de radios hay en cada comuna
radios_metros.COMUNA.value_counts()

In [ ]:
# podemos leer la documentacion y ver si podemos obtenerlo porcentajes
radios_metros.COMUNA.value_counts?

In [ ]:
# asi los tenemos en proporciones
radios_metros.COMUNA.value_counts(normalize = True) 

In [ ]:
# para porcentajes solo tenemos que multiplicar por 100
radios_metros.COMUNA.value_counts(normalize = True) * 100

In [ ]:
#creamos otra variable que sea el % de hogaron con NBI por radio
radios_metros['NBI'] = radios_metros['HOGARES_NBI'] / radios_metros['HOGARES'] * 100
radios_metros.loc[:5,['RADIO_ID','NBI']]

In [ ]:
#obtenemos estadisticos
estadisticos_caba = radios_metros.NBI.describe()
estadisticos_caba

In [ ]:
nbi_caba = estadisticos_caba['mean']
print('El NBI en CABA es de:',nbi_caba)

In [ ]:
#hay relacion entre densidad y NBI ?
sns.regplot(x="NBI", y="densidad", data=radios_metros)

In [ ]:
comuna_4 =  radios_metros.loc[ radios_metros.COMUNA == '4',['RADIO_ID','NBI','geometry']]
comuna_4.plot(column = "NBI",scheme = "Quantiles", k = 5)

In [ ]:
estadisticos_comuna_4 = comuna_4.NBI.describe()
estadisticos_comuna_4

In [ ]:
nbi_comuna_4 = estadisticos_comuna_4['mean']
nbi_comuna_4

In [ ]:
print('El NBI en CABA es de %.2f mientras que en la Comuna 4 es de %.2f' % (nbi_caba,nbi_comuna_4))

In [ ]:
#Podemos agregar algo de logica
if nbi_caba > nbi_comuna_4:
  texto_comparativo = 'ALTO'
else:
  texto_comparativo = 'BAJO'

print('El NBI en CABA es mas %s que el de la Comuna 4, ya que mientras que en CABA es de %.2f, en la Comuna 4 es de %.2f' % (texto_comparativo,nbi_caba,nbi_comuna_4))